In [ ]:
def createADS(workingDir,train):
    os.chdir(workingDir)
    #===========================================================================
    #Import relavent csv's
    #===========================================================================
    receiptsRAW_df=pd.read_csv('201904 sales receipts.csv')
    
    #===========================================================================
    #We only want customers part of the loyalty program
    #===========================================================================
    if train==1:
        receiptsRAW_df=receiptsRAW_df[receiptsRAW_df['customer_id']>0]
    else:
        pass

    NYC_WeatherRAW_df=pd.read_csv('NYC_weather.csv')
    ProductsRAW_df=pd.read_csv('product.csv')
    
    #===========================================================================
    #Columns to keep for each dataframe
    #===========================================================================
    receipts_colNames=['transaction_id','transaction_date','transaction_time','sales_outlet_id','instore_yn','product_id','quantity','line_item_amount']
    Weather_colNames=['date','Avg_temp','Precip_inches_yesterday','wind_mph_avg']
    product_colNames=['product_id','product_group']
    
    #===========================================================================
    #function to drop unnecessary columns
    #===========================================================================
    def keepCols(input_df,cols_to_keep):
        Output_df = input_df[input_df.columns.intersection(cols_to_keep)]
        return Output_df
    
    #===========================================================================
    #Function calls
    #===========================================================================
    receipts_df=keepCols(receiptsRAW_df,receipts_colNames)
    Weather_df=keepCols(NYC_WeatherRAW_df,Weather_colNames)
    products_df=keepCols(ProductsRAW_df,product_colNames)

    #===========================================================================
    #Formatting the weather dataframe
    #===========================================================================
    #Shift precipitation one cell up; This will turn previous day's precipitation value into current day's precipitation
    Weather_df['Precip_inches_yesterday'] = Weather_df['Precip_inches_yesterday'].shift(-1).ffill()
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

    #Rename the column
    Weather_df = Weather_df.rename(columns={'Precip_inches_yesterday': 'Precip_inches'})

    Weather_df['date']=pd.to_datetime(Weather_df['date'])

    #===========================================================================
    #Formatting the receipts dataframe
    #===========================================================================
    #Convert store into dummy columns - one binary column per store
    receipts_df=pd.get_dummies(receipts_df,columns=['sales_outlet_id'], prefix='Store ')

    #Convert Y/N values to 1/0
    receipts_df['instore_yn'] = receipts_df['instore_yn'].apply(lambda x: 1 if x=='Y' else 0)

    receipts_df['transaction_date']=pd.to_datetime(receipts_df['transaction_date'])

    #Whole timestamp is not necessary and will get in the way of kmeans... 
    #The below is extracting the hour and dropping the original column
    receipts_df['Hour']=pd.to_numeric(receipts_df['transaction_time'].str[:2])
    receipts_df.drop('transaction_time', axis=1, inplace=True)  


    #===========================================================================
    #Merging products dataframe with the receipts dataframe
    #===========================================================================

    Receipts_ProdGrp_df=receipts_df.merge(products_df,left_on='product_id',right_on='product_id',how='left')

    #===========================================================================
    #Merging Receipts_ProdGrp_df dataframe with the weather dataframe
    #===========================================================================

    Receipts_ProdWthr_df=Receipts_ProdGrp_df.merge(Weather_df,left_on='transaction_date',right_on='date',how='left')
    Receipts_ProdWthr_df.drop('transaction_date', axis=1, inplace=True)  

    #===========================================================================
    #Seperate and create a pivot out of Transaction ID, product_group, and line_item_amount
    #===========================================================================

    pvt_cols=['transaction_id','line_item_amount','product_group']
    OrderAmt_byGrp_df=keepCols(Receipts_ProdWthr_df,pvt_cols)

    OrderAmt_byGrp_PivotDF = pd.pivot_table(OrderAmt_byGrp_df, values='line_item_amount', index=['transaction_id'],columns=['product_group'], aggfunc=np.sum,fill_value=0)

    #===========================================================================
    #Aggregate other cols on transaction id
    #===========================================================================

    neededCols_forAgg=['transaction_id','instore_yn','quantity','line_item_amount','Store _3','Store _5','Store _8','Hour','Avg_temp','Precip_inches','wind_mph_avg']
    Agg_ADS=keepCols(Receipts_ProdWthr_df,neededCols_forAgg)

    OrdrLvl_ADS=Agg_ADS.groupby('transaction_id').agg({'instore_yn':'max', 'quantity':'sum', 'line_item_amount':'sum', 'Store _3':'max','Store _5':'max','Store _8':'max','Hour':'max','Avg_temp':'max','Precip_inches':'max','wind_mph_avg':'max'})
    OrdrLvl_ADS = OrdrLvl_ADS.rename(columns={'quantity': 'TotalQuantity','line_item_amount': 'TotalAmount'})

    #===========================================================================
    #Merge pivot with above aggregated table
    #===========================================================================

    Kmeans_ADS=OrdrLvl_ADS.merge(OrderAmt_byGrp_PivotDF,left_on='transaction_id',right_on='transaction_id',how='left')

    return Kmeans_ADS

In [ ]:
def selectOptimalCluster_Count(workingDir,Kmeans_ADS,cluster_min,cluster_max):
    os.chdir(workingDir)
    wcss = []
    silhouette_coefficients=[]
    varRatio_list=[]
    for i in range(cluster_min, cluster_max):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, n_init=10, random_state=0)
        kmeans.fit(Kmeans_ADS)
        wcss.append(kmeans.inertia_)
        score = silhouette_score(Kmeans_ADS, kmeans.labels_, metric = 'euclidean')
        scoreDict={i:score}
        silhouette_coefficients.append(scoreDict)

    #print(wcss)
    plt.plot(range(cluster_min, cluster_max), wcss,'bx-')
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.xticks(np.arange(1, cluster_max+1, 1))
    #plt.savefig('ElbowPlot.png')
    plt.show()



In [ ]:
def Run9Clusters(workingDir,Kmeans_ADS):
    os.chdir(workingDir)
    #===========================================================================
    #Best number of clusters was 9. The below runs the model for 9 clusters
    #===========================================================================
    #If the kmeans output dataframe exists and has content in it, delete it
    Kmeans_ADS_wClusters=None
    if Kmeans_ADS_wClusters is not None:
        del Kmeans_ADS_wClusters

    #Run the kmeans
    kmeans = KMeans(n_clusters=9, init='random', max_iter=500, n_init=10, random_state=0)

    #===========================================================================
    #Generate the predictions
    #===========================================================================
    pred_y = kmeans.fit_predict(Kmeans_ADS)
    #===========================================================================
    #Create a copy of the ADS to train the regression analysis
    #===========================================================================
    Reg_TrainDF=Kmeans_ADS.copy()
    #===========================================================================
    #Insert the predictions as a new column 
    #(and change DF name in preparation of regression)
    #===========================================================================
    Reg_TrainDF['Cluster']=pred_y

    #===========================================================================
    #For readibility, start cluster numbers at 1 and go to 9
    #===========================================================================
    Reg_TrainDF['Cluster'] = Reg_TrainDF['Cluster'].apply(lambda x : x+1)

    #===========================================================================   
    #Turn transaction_id into column again
    #===========================================================================
    Reg_TrainDF['transaction_id'] = Reg_TrainDF.index

    #===========================================================================
    #Reset index to default
    #===========================================================================
    Reg_TrainDF.reset_index(inplace = True,drop=True) 
    
    #===========================================================================
    #Export results for analysis
    #===========================================================================
    #Reg_TrainDF.to_csv('PyCluster.csv',index=False) 
    return Reg_TrainDF

In [ ]:
def CreateBubbleChart(workingDir,BubbleChart_source):
    os.chdir(workingDir)
    #===========================================================================
    #Create bubble chart
    #===========================================================================    
    #Create a copy of the ADS for the purposes of this plot
    
    BubbleChart_source=BubbleChart_source[['transaction_id','TotalAmount','Cluster']]

    #===========================================================================
    #Re-import Data
    #===========================================================================
    receiptsRAW_df=pd.read_csv('201904 sales receipts.csv')

    #===========================================================================
    #For this plot, we only need transaction ID and customer ID from original data
    #===========================================================================
    receipts_colNames=['transaction_id','customer_id']
    receipts_df = receiptsRAW_df[receiptsRAW_df.columns.intersection(receipts_colNames)]
    receipts_df=receipts_df[receipts_df['customer_id']>0]

    #===========================================================================
    #This table needs to be unique at an order level
    #===========================================================================
    OrderCust_df=receipts_df.drop_duplicates()

    #===========================================================================
    #Merge the tables to derive a table unique at cluster level
    #===========================================================================
    BubbleChart_source=BubbleChart_source.merge(OrderCust_df,left_on='transaction_id',right_on='transaction_id')
    BubbleChart_source=BubbleChart_source.groupby(by='Cluster',as_index=False).agg({'TotalAmount':'sum','transaction_id':'count','customer_id':'count'})
    BubbleChart_source = BubbleChart_source.rename(columns={'TotalAmount':'Total_Sales','transaction_id': 'Count_of_Orders','customer_id': 'Count_of_Customers'})

    #===========================================================================
    #Plot the table
    #===========================================================================
    #Colors to use for the plot
    colors=['red','blue','cyan','green','magenta','gray','yellow','orange','lime']


    fig, ax = plt.subplots(figsize=(10,10))

    #===========================================================================
    #Derive the unique list of cluster numbers
    #===========================================================================
    Clust=list(dict.fromkeys(BubbleChart_source['Cluster']))

    #===========================================================================
    #For each cluster filter the table and plot the row
        #X-axis = Number of orders
        #Y-axis = Sales
        #Size = Count of customers
    #===========================================================================

    i=0
    while i<=(len(Clust)-1):
        #===========================================================================
        #Store the product category as a string in a variable...
        #===========================================================================
        Clust_toPlt=Clust[i]

        #===========================================================================
        #Filter the data for that product category
        #===========================================================================
        plot_df=BubbleChart_source[BubbleChart_source['Cluster'] == Clust_toPlt]

        #===========================================================================
        #Create a scatter plot with that filtered data
            #Note: The next iteration, the next "scatter plot", will just get added 
            #      to the previous iteration's, plot.
        #===========================================================================
        ax.plot(plot_df['Count_of_Orders'],plot_df['Total_Sales'], 'o',label=Clust_toPlt,markersize=plot_df['Count_of_Customers']/80)
        i=i+1

    #===========================================================================
    #Show the legend
    #===========================================================================
    lgnd=ax.legend(bbox_to_anchor=(1,0.8), title="Clusters",markerscale=0.15)

    #===========================================================================
    #Set the axes
    #===========================================================================
    ax.set_ylabel('Sales') 
    ax.set_xlabel('Number of Orders')
    ax.set_title('Who Should we Target') 

    #===========================================================================
    #Format the axes to show as dollar values
    #===========================================================================
    ax.yaxis.set_major_formatter('${x:1.2f}')

    plt.savefig('TargetSegments_Bubble.png')
    plt.show()


In [ ]:
def RunRegression(workingDir,Reg_TrainDF,Reg_TestDF):
    os.chdir(workingDir)
    #===========================================================================
    #Regression analysis - Can we predict cluster?
    #===========================================================================
    Reg_TestDF.fillna(0)
    Reg_TestDF['transaction_id'] = Reg_TestDF.index
    Reg_TestDF.reset_index(inplace = True,drop=True)   
    
    
    #Merge with train to bring in cluster column
    Reg_TestDF = pd.merge(Reg_TestDF,Reg_TrainDF[['transaction_id','Cluster']],on='transaction_id', how='left')
    Reg_TestDF=Reg_TestDF.fillna(0)

    #What are the variables we are using to predict cluster
    attributes=Reg_TrainDF[['instore_yn', 'TotalQuantity', 'TotalAmount', 'Store _3', 'Store _5', 'Store _8', 'Hour','Avg_temp', 'Precip_inches', 'wind_mph_avg', 'Add-ons', 'Beverages', 'Food','Merchandise', 'Whole Bean/Teas']].values
    #Y-variable. The variable we are trying to predict
    Y_var=Reg_TrainDF[['Cluster']].values
    #Create and run the model
    regressor = LinearRegression()  
    regressor.fit(attributes, Y_var)

    #The coefficients of the model
    coeff_df = pd.DataFrame(regressor.coef_)
    coeff_df.columns = ['instore_yn', 'TotalQuantity', 'TotalAmount', 'Store _3', 'Store _5', 'Store _8', 'Hour','Avg_temp', 'Precip_inches', 'wind_mph_avg', 'Add-ons', 'Beverages', 'Food','Merchandise', 'Whole Bean/Teas']
    print("Intercept:",regressor.intercept_)


    #Predict members' cluster (see if the model is accurately predicting)
    TrainCluster_pred = regressor.predict(Reg_TrainDF[['instore_yn', 'TotalQuantity', 'TotalAmount', 'Store _3', 'Store _5', 'Store _8', 'Hour','Avg_temp', 'Precip_inches', 'wind_mph_avg', 'Add-ons', 'Beverages', 'Food','Merchandise', 'Whole Bean/Teas']])
    Reg_TrainDF['Predicted_Cluster']=TrainCluster_pred.astype(int)

    print("R^2: ",metrics.r2_score(Y_var, TrainCluster_pred))
    print('Mean Absolute Error:', metrics.mean_absolute_error(Y_var, TrainCluster_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(Y_var, TrainCluster_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_var, TrainCluster_pred)))
    print("Maxiumum Value: ", max(TrainCluster_pred))
    #Reg_TrainDF.to_csv('Reg_DF',index=False) <-Insert path to export to here


    #Conclusion: No relationship between order details and customer information (who/when/how a customer buys a product)


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import metrics
import datetime as dt
import os
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


pd.options.mode.chained_assignment = None  # default='warn'
def runEverything(workingDir):
    KMeansADS=None
    Reg_TestDF=None
    KMeansADS=createADS(workingDir,1)
    Reg_TestDF=createADS(workingDir,0)
    print("ADS created")
    #selectOptimalCluster_Count(workingDir,KMeansADS,5,25)
    #print("Optimal Selected")
    BubbleSource= Run9Clusters(workingDir,KMeansADS)
    print("9 Clusters made")
    CreateBubbleChart(workingDir,BubbleSource)
    print("Bubble Chart made")
    RunRegression(workingDir,BubbleSource,Reg_TestDF)
    print("Regression Complete")
print("Please input the directory where the RAW files are stored: ")
directory_str=input()
print("Directory: "+ directory_str)
directory_str=directory_str+"\\"
runEverything(directory_str)